In [3]:
"""=============================================================================
Embed all images into a matrix Z1.
Select a gene g2i at random, embed it to vector z2i.
Find the z1 in Z1 that is closest to z2i.
It should either be the image pair for g2i or share similar image properties.
============================================================================="""

%cd /Users/gwg/local/dmcm
%matplotlib inline

/Users/gwg/local/dmcm


In [9]:
import random

from   sklearn import manifold, decomposition 
from   sklearn import manifold, decomposition 
from   scipy.spatial.distance import cdist 
import numpy as np

import matplotlib 
import matplotlib.pyplot as plt 
from   matplotlib.offsetbox import OffsetImage, AnnotationBbox

import torch 
from   torch.autograd import Variable 
from   torchvision.utils import save_image

from   models import load_trained_model 
from   data.synth.config import SynthConfig 
from   data import SynthDataset

config  = SynthConfig()
dataset = SynthDataset(config)
base  = '/Users/gwg/local/dmcm/experiments/20180304_synth'
dir_  = 'synth_l1_coef-0.0001_lr-0.0001'
fname = '%s/%s/model.pt' % (base, dir_)
model = load_trained_model(config, fname, cpu=True)
model = model.eval()

X1 = dataset.images
X2 = dataset.colors

Z1 = torch.load('%s/%s/K1.pt' % (base, dir_)).numpy()
Z2 = torch.load('%s/%s/K2.pt' % (base, dir_)).numpy()

In [ ]:
# Select random image, color pair.
r = random.randint(0, len(dataset))
x1, x2, _ = dataset[r]
x1        = Variable(x1)
x2        = Variable(x2)

(z1, x1_recon, mu, logvar), z2 = model.forward([x1, x2])

# Sort the color vectors based on the reference color vector.
ref = z2.data.numpy()
def euclidean(x):
    x1, x2 = x
    return np.linalg.norm(ref - x1)

# Sort color vector embeddings and original images by reference color vector.
Z2s, X1s = zip(*sorted(zip(Z2, X1), key=euclidean))
# Skip first one, which is reference image.
X1s = X1s[1:21]

DIR = '/Users/gwg/local/dmcm/scratch/nearest/'
save_image(x1.data, '%s/0_reference.png' % DIR)
for i, x in enumerate(X1s):
    save_image(x, '%s/%s.png' % (DIR, i))